# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
T. Henning  ->  T. Henning  |  ['T. Henning']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Li  ->  J. Li  |  ['J. Li']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']


Arxiv has 115 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.06394


extracting tarball to tmp_2512.06394... done.
Retrieving document from  https://arxiv.org/e-print/2512.06439


extracting tarball to tmp_2512.06439...

 done.


...

extracting tarball to tmp_2512.07236...

 done.
Retrieving document from  https://arxiv.org/e-print/2512.07300


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2512.07236/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'define.tex' from 'tmp_2512.07236/define.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2512.07300...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.06439-b31b1b.svg)](https://arxiv.org/abs/2512.06439) | **Water vapor emission at the warm cavity wall of the HD 100546 disk as revealed by ALMA**  |
|| L. Rampinelli, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-12-09*|
|*Comments*| *13 pages, 6 figures. Accepted for publication in ApJ Letters December 5, 2025*|
|**Abstract**|            We present spatially resolved ALMA observations of the water line at 183 GHz in the disk around the Herbig star HD 100546. The water vapor emission peaks at the inner edge of the warm dust cavity, located ~15 au from the central star. We attribute this to thermal desorption at the water snowline, shifted outward at the dust cavity wall directly heated by the intense radiation. This represents the first spatially resolved image of the water snowline using ALMA observations of the main water isotopologue in a protoplanetary disk. The water emission morphology peaking inside the first dust ring is consistent with previous ALMA detections of oxygen-bearing complex organic molecules in the disk, including thermally desorbed methanol. These findings signal that warm cavities of transition disks provide ideal targets to directly reconstruct the spatial distribution of water vapor and the snowline location with ALMA, and directly connect water vapor emission to ice desorption of complex organic species.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.06731-b31b1b.svg)](https://arxiv.org/abs/2512.06731) | **Revisiting the atmosphere of HAT-P-70b with CARMENES high-resolution transmission spectroscopy**  |
|| T. Gan, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-12-09*|
|*Comments*| *19 pages, 11 figures, 3 tables, accepted for publication in AJ*|
|**Abstract**|            Owing to hot and inflated envelopes that facilitate atmospheric studies, ultra-hot Jupiters (UHJs) have attracted much attention. Significant progress has been achieved, from enlarging the sample size to broadening the studies to encompass diverse stellar types and ages. Here, we present a transmission spectroscopy study of HAT-P-70b, an UHJ orbiting a young A-type star, through high-resolution observations with CARMENES at the 3.5m Calar Alto telescope. By using the line-by-line technique, we confirm the previous detections of Ha, Na I, and Ca II, report a new tentative detection of K I, and impose an upper limit on the He triplet absorption. Through cross-correlation analysis, we identify the Ca II and Fe I absorptions, both blue-shifted by approximately 5 km/s, indicating a day-to-night side atmospheric wind. Additionally, we find a new tentative detection of K I. We do not see any significant atmospheric molecular signal in the near-infrared data. Putting HAT-P-70b in the context of UHJs from the literature, it turns out that (1) Ha absorption is more common on gas giants orbiting stars younger than 1 Gyr, with a relative detection probability of $P_{\rm Age<1\,Gyr}({\rm Ha})/P_{\rm Age\geq1\,Gyr}({\rm Ha})\sim 3$; (2) any UHJ is likely to exhibit Fe I absorption if it has Ca II.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.06394-b31b1b.svg)](https://arxiv.org/abs/2512.06394) | **Rebrightenings of gamma-ray burst afterglows from an increasing magnetic inclination angle of a nascent magnetar**  |
|| M. Xu, <mark>J. Li</mark>, C. F. Xiao, H. H. Qiu |
|*Appeared on*| *2025-12-09*|
|*Comments*| *7 pages, 3 figures, 1 table, A&A, in press*|
|**Abstract**|            A nascent magnetar, accompanying a gamma-ray burst (GRB) explosion, releases enormous rotational energy via magnetic dipole radiation. The energy loss rate of the magnetar is determined by the strength of the magnetic field at the pole. We investigated the effect of the magnetic inclination angle on the energy loss rate. The released energy is injected into the GRB jet and shapes the light curves of GRB afterglow. Different evolutionary approaches lead to different curves shapes.A shallow decay phase in GRB X-ray afterglow may result from energy injection from a magnetar with a fixed inclination angle. A two-plateau phase may result from a decreasing inclination angle scenario. In this study, we considered an increasing inclination angle scenario. The energy loss rate of the magnetar increases as the magnetic inclination angle grows. Our analysis reveals that as the lost rotational energy injected into the GRB jet increases, rebrightening phases occur in the GRB afterglows. The rebrightening features are slight and short-lived. The observed afterglow rebrightening of GRB 170822A and GRB 230414B can be well explained within our framework. Some GRB X-ray afterglows that exhibit slight and early rebrightenings may result from an increasing magnetic inclination angle of a nascent magnetar.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.07233-b31b1b.svg)](https://arxiv.org/abs/2512.07233) | **EP241217a: a likely Type II GRB with an achromatic bump at z = 4.59**  |
|| H. Zhou, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-12-09*|
|*Comments*| *26 pages, 10 figures, 4 tables*|
|**Abstract**|            EP241217a is an X-ray transient detected by the Einstein Probe (EP) lasting for about 100 seconds and without accompanying $\gamma$-ray detection. The optical spectroscopy reveals the redshift of EP241217a is 4.59. By combining the $\gamma$-ray upper limit provided by GECAM-C, there is a considerable possibility that EP241217a is a typical Type II gamma-ray burst (GRB), but it is fainter than the detection threshold of any available $\gamma$-ray monitors (i.e., $E_{\gamma,{\rm iso}}\lesssim10^{53}$ erg). The X-ray light curve exhibits a plateau lasting for $\sim5\times10^4$ seconds. However, the joint analysis with optical data suggests the presence of an achromatic bump peaking at $\sim3\times10^4$ s after the trigger, indicating the actual duration of the X-ray plateau may be significantly shorter than it appears. To interpret the achromatic bump, we adopt the scenario of a mildly relativistic jet coasting in a wind-like medium and encountering a rapid density enhancement of the circumburst medium, which is likely induced by the the interaction of the progenitor's stellar wind and the interstellar medium. However, this model cannot fully explain observed data, and some issues do exist, e.g., the observed spectrum is harder than the model prediction. Consequently, we conclude that the scenario of a mildly relativistic jet coasting in the wind-like medium cannot explain all observed features of EP241217a. In addition, some alternative models commonly invoked to explain X-ray plateaus are discussed, but there are more or less issues when they are applied to EP241217a. Therefore, further theoretical modeling is encouraged to explore the origin of EP241217a.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.07236-b31b1b.svg)](https://arxiv.org/abs/2512.07236) | **Counting voids and filaments: Betti Curves as a Powerful Probe for Cosmology**  |
|| <mark>J. Li</mark>, C. Zhao |
|*Appeared on*| *2025-12-09*|
|*Comments*| *22 pages, 15 figures*|
|**Abstract**|            Topological analysis of galaxy distributions has gathered increasing attention in cosmology, as they are able to capture non-Gaussian features of large-scale structures (LSS) that are overlooked by conventional two-point clustering statistics. We utilize Betti curves, a summary statistic derived from persistent homology, to characterize the multiscale topological features of the LSS, including connected components, loops, and voids, as a complementary cosmological probe. Using halo catalogs from the \textsc{Quijote} suite, we construct Betti curves, assess their sensitivity to cosmological parameters, and train automated machine learning based emulators to model their dependence on cosmological parameters. Our Bayesian inference recovers unbiased estimation of cosmological parameters, notably $n_{\mathrm{s}}$, $\sigma_8$, and $\Omega_{\mathrm{m}}$, while validation on sub-box simulations confirms robustness against cosmic variance. We further investigate the impact of redshift-space distortions (RSD) on Betti curves and demonstrate that including RSD enhances sensitivity to growth-related parameters. By jointly analyzing Betti curves and the power spectrum, we achieve significantly tightened constraints than using power spectrum alone on parameters such as $n_{\mathrm{s}}$, $\sigma_8$, and $w$. These findings highlight Betti curves -- especially when combined with traditional two-point statistics -- as a promising, interpretable tool for future galaxy survey analyses.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.07300-b31b1b.svg)](https://arxiv.org/abs/2512.07300) | **Back-End System of BURSTT**  |
|| K.-Y. Lin, et al. -- incl., <mark>H. Jiang</mark> |
|*Appeared on*| *2025-12-09*|
|*Comments*| *submitted to PASP*|
|**Abstract**|            The Bustling Universe Radio Survey Telescope in Taiwan (BURSTT) is a new-generation wide-angle radio telescope specifically designed to survey Fast Radio Bursts (FRBs), energetic millisecond-duration pulses of unknown extragalactic origin. To realize its scientific potential, which includes detecting approximately 50 FRBs per year and sub-arcsecond localization capability, the system is designed to perform real-time beamforming and pulse search over the \SI{60}{\degree} $\times$ \SI{120}{\degree} field of view. This paper provides a detailed account of the design, implementation, and performance validation of the BURSTT back-end System. The system employs an efficient multi-stage processing architecture: initial beamforming is executed on the Xilinx ZCU216 RF System-on-Chip (RFSoC) platform; data is then transferred to Intel Xeon servers, where AVX-512 and AMX instruction sets are utilized for the second stage of beamforming and channelization, achieving high computational efficiency to ensure real-time capability. A highly optimized \texttt{bonsai} de-dispersion algorithm performs a real-time pulse search and triggering across 256 beams, which, upon detection, issues commands to the distributed outrigger system to save voltage data for very-long baseline interferometry (VLBI) precise localization. System performance has been validated through beamforming tests using bright radio sources and real-time detection of known pulsars, confirming the high fidelity of the signal processing pipeline.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.06439/./radial_profile.png', 'tmp_2512.06439/./spectra.png', 'tmp_2512.06439/./model_water_ncr.png']
copying  tmp_2512.06439/./radial_profile.png to _build/html/
copying  tmp_2512.06439/./spectra.png to _build/html/
copying  tmp_2512.06439/./model_water_ncr.png to _build/html/
exported in  _build/html/2512.06439.md
    + _build/html/tmp_2512.06439/./radial_profile.png
    + _build/html/tmp_2512.06439/./spectra.png
    + _build/html/tmp_2512.06439/./model_water_ncr.png
found figures ['tmp_2512.06731/./FigSL/HAT-P-70b_Ha_He.png', 'tmp_2512.06731/./FigSL/Rp-Age_Period_plot.png', 'tmp_2512.06731/./FigSL/HAT-P-70_Na_K.png']
copying  tmp_2512.06731/./FigSL/HAT-P-70b_Ha_He.png to _build/html/
copying  tmp_2512.06731/./FigSL/Rp-Age_Period_plot.png to _build/html/
copying  tmp_2512.06731/./FigSL/HAT-P-70_Na_K.png to _build/html/
exported in  _build/html/2512.06731.md
    + _build/html/tmp_2512.06731/./FigSL/HAT-P-70b_Ha_He.png
    + _build/html/tmp_2512.06731/./FigSL/Rp

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km~s^{-1} }$
$\newcommand$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Water vapor emission at the warm cavity wall of the HD 100546 disk as revealed by ALMA

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.06439-b31b1b.svg)](https://arxiv.org/abs/2512.06439)<mark>Appeared on: 2025-12-09</mark> -  _13 pages, 6 figures. Accepted for publication in ApJ Letters December 5, 2025_

</div>
<div id="authors">

R. Luna, et al.

</div>
<div id="abstract">

**Abstract:** We present spatially resolved ALMA observations of the water line at 183 GHz in the disk around the Herbig star HD 100546. The water vapor emission peaks at the inner edge of the warm dust cavity, located $\sim15$ au from the central star. We attribute this to thermal desorption at the water snowline, shifted outward at the dust cavity wall directly heated by the intense radiation. This represents the first spatially resolved image of the water snowline using ALMA observations of the main water isotopologue in a protoplanetary disk. The water emission morphology peaking inside the first dust ring is consistent with previous ALMA detections of oxygen-bearing complex organic molecules in the disk, including thermally desorbed methanol.These findings signal that warm cavities of transition disks provide ideal targets to directly reconstruct the spatial distribution of water vapor and the snowline location with ALMA, and directly connect water vapor emission to ice desorption of complex organic species.

</div>

<div id="div_fig1">

<img src="tmp_2512.06439/./radial_profile.png" alt="Fig2" width="100%"/>

**Figure 2. -** Radial profile of the integrated intensity of the water emission (blue line) compared to the continuum one (dashed black line). Blue ribbons show the uncertainty on the water radial profile. The light blue region indicates the radial extent of the 3$\mu$m water ice absorption band from Gemini observations \citep{honda2016}, and the vertical blue line shows the inner edge of the water emission inferred from _Herschel_ observations \citep[]{vandishoeck2021,pirovano2022}. The hatched region corresponds to the beam semi-major axis of the water cube used to extract the profile. The negative trend of the radial profile beyond 100 au seems to be an artifact due to negative sidelobes in the PSF. (*fig:radial_profile*)

</div>
<div id="div_fig2">

<img src="tmp_2512.06439/./spectra.png" alt="Fig3" width="100%"/>

**Figure 3. -** Disk-integrated spectrum of the \ce{H2O} line, extracted from a de-projected circle with a radius of 100 au (left panel), and after correcting for the Keplerian rotation (right panel). The vertical red dashed line indicates the systemic velocity of $5.7$ $\kms$, while the blue line on the top right side of each panel is the uncertainty. The dip in the spectrum in the left panel at $\sim-21$ $\kms$ is due to the low transmission at the peak of the atmospheric telluric water line (see Appendix \ref{app:noise} for details on the bandpass calibration and its effect on the retrieved water spectra). (*fig:spectrum*)

</div>
<div id="div_fig3">

<img src="tmp_2512.06439/./model_water_ncr.png" alt="Fig6" width="100%"/>

**Figure 6. -** Left: critical density as a function of temperature for the _Herschel_ lines included in \cite{pirovano2022}(solid lines) and the ALMA 183 GHz line (dashed). The colorbar shows the upper state energies of each transition. Right: Water column density from the DALI model of the HD 100546 disk from \cite{leemker2024}. The pink horizontal line indicates the density above which water self-shielding is expected to be important. The brown shaded region indicate the dust ring between 20 and 40 au. (*fig:model_water_ncr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.06439"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\todo}[1]{\textcolor{red}{#1}}$
$\newcommand{\rev}[1]{\textcolor{blue}{#1}}$
$\newcommand{\code}[1]{{\texttt{#1}}}$
$\newcommand{\tess}{{\it TESS}}$
$\newcommand{\kepler}{{\it Kepler}}$
$\newcommand{\LCOGT}{{\it LCOGT}}$
$\newcommand{\gaia}{{\it Gaia}}$
$\newcommand{\wise}{{\it WISE}}$
$\newcommand{\jwst}{{\it JWST}}$
$\newcommand{\elt}{{\it ELT}}$
$\newcommand{\tar}{{HAT-P-70}}$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{\theHfigure}{A.\arabic{figure}}$
$\newcommand{\theHfigure}{A.\arabic{figure}}$
$\newcommand{\theHfigure}{A.\arabic{figure}}$
$\newcommand{\theHfigure}{A.\arabic{figure}}$</div>



<div id="title">

# Revisiting the atmosphere of HAT-P-70b with CARMENES high-resolution transmission spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.06731-b31b1b.svg)](https://arxiv.org/abs/2512.06731)<mark>Appeared on: 2025-12-09</mark> -  _19 pages, 11 figures, 3 tables, accepted for publication in AJ_

</div>
<div id="authors">

Tianjun~Gan, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Owing to hot and inflated envelopes that facilitate atmospheric studies, ultra-hot Jupiters (UHJs) have attracted much attention. Significant progress has been achieved, from enlarging the sample size to broadening the studies to encompass diverse stellar types and ages. Here, we present a transmission spectroscopy study of $\tar$ b, an UHJ orbiting a young A-type star, through high-resolution observations with CARMENES at the 3.5 m Calar Alto telescope. By using the line-by-line technique, we confirm the previous detections of H $\alpha$ , Na ${\sc i}$ , and Ca ${\sc ii}$ , report a new tentative detection of K ${\sc i}$ , and impose an upper limit on the He triplet absorption. Through cross-correlation analysis, we identify the Ca ${\sc ii}$ and Fe ${\sc i}$ absorptions, both blue-shifted by approximately $\rm 5 km s^{-1}$ , indicating a day-to-night side atmospheric wind. Additionally, we find a new tentative detection of K ${\sc i}$ . We do not see any significant atmospheric molecular signal in the near-infrared data. Putting $\tar$ b in the context of UHJs from the literature, it turns out that (1) H $\alpha$ absorption is more common on gas giants orbiting stars younger than 1 Gyr, with a relative detection probability of $P_{\rm Age<1 Gyr}({\rm H}\alpha)/P_{\rm Age\geq1 Gyr}({\rm H}\alpha)\sim 3$ ; (2) any UHJ is likely to exhibit Fe ${\sc i}$ absorption if it has Ca ${\sc ii}$ .

</div>

<div id="div_fig1">

<img src="tmp_2512.06731/./FigSL/HAT-P-70b_Ha_He.png" alt="Fig7" width="100%"/>

**Figure 7. -**  Residual maps and transmission spectra around H$\alpha$(_left_) and the He {\sc i} NIR triplet (_right_) lines. _Top panels_: Residual maps in the stellar rest frame. The time since mid-transit time ($T_c$) is shown on the vertical axis, wavelength is on the horizontal axis, and relative absorption is color-coded. Dashed blue horizontal lines indicate the transit duration. Dashed cyan tilted lines show the theoretical trace of the planetary signals. _Bottom panels_: Transmission spectra obtained combining all the spectra between the first and fourth contacts. We show the original data in light gray and the data binned by 0.2 Å in black. The best Gaussian fit model is shown in red along with its $1\sigma$ uncertainties (shaded red region). Dashed cyan vertical lines indicate the H$\alpha$(_left_) and the He {\sc i} triplet (_right_) lines positions. All wavelengths in this figure are given in vacuum.
     (*Fig: TS Halpha Helium*)

</div>
<div id="div_fig2">

<img src="tmp_2512.06731/./FigSL/Rp-Age_Period_plot.png" alt="Fig12" width="100%"/>

**Figure 12. -** 
    Distribution of H$\alpha$ observations for gas giants as function of stellar age vs. planetary radius (top panel), and period vs. radius diagram for $<$1 Gyr-old (bottom left panel) and $\ge$1 Gyr-old (bottom right panel) planets. $\tar$ b is labeled and marked with a blue star in the panels. Planets with detection of H$\alpha$ are marked with blue dots, while those with H$\alpha$ non-detections/upper limits are marked with red crosses. We indicate the 1 Gyr value with a dashed black horizontal line in the top panel. Results are retrieved from the ExoAtmospheres website.
     (*Fig: Age - Rp Halpha*)

</div>
<div id="div_fig3">

<img src="tmp_2512.06731/./FigSL/HAT-P-70_Na_K.png" alt="Fig8" width="100%"/>

**Figure 8. -**  Same as Figure \ref{Fig: TS Halpha Helium} but for the Na {\sc i} doublet lines (left panels) and K {\sc i}$\lambda$7701$\angstrom$(right panels). The white vertical bands are the masked regions due to interstellar medium absorption. Only a fit for Na {\sc i} D2 is presented. A shallow dip can be seen in the result of K {\sc i}, which might be due to atmospheric absorption.
     (*Fig: TS Na Doublet*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.06731"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

138  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
